# 普通名詞の獲得
本ノートブックでは普通名詞の獲得について仮説を立てて検証する。

最初に検証する仮説はこちら:

* 辞書ベースでラベリングした際、正解部分以外にラベリングした場合は普通名詞である可能性が高い


まずは必要なモジュールをインポートする。

In [18]:
import os

from seqeval.metrics import f1_score

from entitypedia.evaluation.converter import to_iob2
from entitypedia.knowledge_ner.recognizer import KnowledgeBaseRecognizer
from entitypedia.corpora.wikipedia.extractor import load_jsonl

まずは辞書ベースの分類器を読み込む。

In [19]:
recognizer = KnowledgeBaseRecognizer()
file_path = '../data/interim/title_entity.jsonl'
for d in load_jsonl(file_path):
    entity, label = list(d.items())[0]
    sub_type = label.split('/')[-1]
    label = label.split('/')[0]
    recognizer.add_entity(entity, label)
    recognizer.add_word(entity, sub_type, page_url='', image_url='')
recognizer.build()

除外するタイプを定義する。

In [20]:
remove_types = {'volume', 'period_date', 'percent', 'url', 'service', 'multiplication',
                'n_person', 'school_age', 'seismic_intensity', 'period_month',
                'phone_number', 'rank', 'n_animal', 'countx_other', 'point',
                'periodx_other', 'calorie', 'space', 'period_time', 'n_country',
                'n_product', 'numex_other', 'latitude_longtitude', 'id_number',
                'n_flora', 'facility_part', 'temperature', 'weight', 'age', 'water_root',
                'n_natural_object_other', 'intensity', 'time', 'n_facility',
                'n_organization', 'postal_address', 'period_year', 'ordinal_number',
                'physical_extent', 'speed', 'measurement_other', 'seismic_magnitude',
                'n_event', 'period_week', 'frequency', 'ignored', 'stock', 'n_location_other'}

次に正解データを読み込む。

In [21]:
BASE_DIR = '../data/raw/corpora'
mainichi_dir = os.path.join(BASE_DIR, 'mainichi')
X_true, y_true = to_iob2(mainichi_dir, remove_types)

正解データを読み込んだら、辞書ベース分類器を用いて、ラベル付する。

In [22]:
y_pred = [recognizer.iob2(text) for text in X_true]

以上で仮説検証の準備ができた。ここからは実際に検証していく。

まずは正解ラベルが付いていない箇所にラベル付している部分を洗い出す。

In [23]:
for y_p, y_t, char in zip(y_pred[0], y_true[0], X_true[0]):
    if y_t == 'O' and y_p != 'O':
        print('{}\t{}'.format(char, y_p))

国	B-other
際	I-other
な	B-city
ど	B-company
点	B-weapon
ビ	B-company
ジ	I-company
ョ	I-company
ン	I-company
の	B-city
基	B-position_vocation
本	I-position_vocation
す	B-position_vocation
は	B-city
の	B-city
会	I-position_vocation
見	I-position_vocation
で	B-company
と	B-position_vocation
す	B-position_vocation
る	B-position_vocation
所	B-company
を	B-position_vocation
発	B-other
表	I-other
し	B-company
た	I-company
月	B-astral_body_other
中	B-weapon
に	B-city
を	B-position_vocation
む	B-city
グ	B-company_group
ル	I-company_group
ー	I-company_group
プ	I-company_group
が	B-insect
発	B-other
表	I-other
す	B-position_vocation
る	B-position_vocation
の	B-city
基	B-position_vocation
本	I-position_vocation
的	B-drug
な	B-city
え	B-position_vocation
を	B-position_vocation
し	B-company
た	I-company
も	B-city
の	B-city
わ	B-river
が	B-insect
国	B-country
に	B-city
ふ	B-position_vocation
さ	B-weapon
わ	I-weapon
し	B-weapon
い	I-weapon
国	B-other
際	I-other
に	B-city
よ	B-city
る	B-position_vocation
世	B-domestic_region
界	I-domestic_region
平	B-war
和	I-war
の	B-ci

上記を分析すると以下のことがわかる。

* 曖昧さ回避ページが多い
* 全体的には普通名詞であることが多い
* 固有名詞の場合もある(e.g. 新 -> 昔の中国の国名)

仮説としては悪くなさそうなので、さらに検証を進めていく。
上記は文字単位でついているので、単語単位に変換して分析する。

In [24]:
from seqeval.metrics.sequence_labeling import get_entities

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

In [25]:
words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
words[0:10]

['国際', 'な', 'ど', '点', 'ビジョン', 'の', '基本', 'す', 'は', 'の']

ここまでで、ラベル付結果を単語単位に変換しました。

得られた単語の統計情報を確認してみます。確認する統計情報は以下の通り。

* トークン数(延べ単語数)
* タイプ数(異なり語数)
* 各単語の頻度

In [26]:
from collections import Counter

freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1702430
Num of types  : 22852
1223686 / 1702430
の	113730
に	67129
を	57958
は	54075
い	52871
が	52026
る	49761
と	49416
で	48583
た	44416
て	44150
な	35477
し	28469
っ	26763
れ	24690
か	24502
ら	22181
も	21313
す	19664
だ	15968
した	15784
り	15429
こ	15266
う	15244
ま	13846
き	12148
く	11936
め	11249
さ	11151
け	10775
ど	10729
あ	10238
え	8419
そ	7845
一	7150
ん	7149
や	7124
人	6737
つ	6645
年	6593
わ	5647
時	5123
大	4880
中	4430
ち	4307
ろ	4157
ば	4132
分	4074
的	3968
み	3836
よう	3803
よ	3728
日	3638
せ	3590
行	3467
お	3294
ず	3110
じ	3020
べ	2938
へ	2875
本	2781
高	2686
間	2676
目	2607
方	2446
げ	2435
しい	2356
要	2337
はな	2322
対	2308
内	2265
生	2177
ほ	2144
場	2128
取	2079
受	2052
力	2042
強	2039
下	2032
初	1922
月	1912
国	1900
新	1866
勝	1865
通	1844
実	1836
とも	1834
地	1816
午	1792
調	1775
む	1737
合	1727
手	1712
び	1690
金	1684
半	1679
かし	1676
現在	1640
話	1627
自	1624
千	1593
数	1592
明	1574
元	1568
円	1547
全	1536
せん	1534
点	1510
問題	1447
表	1447
体	1425
面	1377
ご	1369
また	1349
交渉	1323
連	1320
向	1304
政府	1295
なかった	1266
書	1259
かな	1255
私	1255
度	1243
ぶ	1214
子	1205
より	1199
支	119

長い間	19
スケジュール	19
全国大会	19
共和国	19
大方	19
出来高	19
歳出	19
暴落	19
伝説	19
であい	19
第三者	19
大物	19
木材	19
離脱	19
ドーム	19
聖	19
めし	19
島田	19
時間帯	19
容量	19
土砂	19
信頼性	19
盾	19
ドライバー	19
遺跡	19
解剖	19
期日	19
家賃	19
ガキ	19
ドラフト	19
峰	19
推測	19
復興計画	19
オープン戦	19
朝日	19
表記	19
利子	19
池田市	19
法律相談	19
カラ	19
配当	19
コウ	19
違法性	19
財閥	19
オフレコ	19
誠実	18
粉	18
独身	18
人道	18
パフォーマンス	18
やな	18
プレート	18
顧問	18
近代化	18
版画	18
幹線	18
国土	18
いであ	18
年長	18
わが家	18
通学	18
相模台	18
仙台	18
和歌山	18
新潟	18
鐘	18
冷凍	18
燃料	18
乗り換え	18
本番	18
先住民	18
裸	18
食堂	18
行進	18
ジャー	18
つなぐ	18
肥料	18
名所	18
在学	18
土壌	18
後遺症	18
田舎	18
洋服	18
オフィス	18
風呂	18
セリフ	18
青森	18
河川敷	18
海外旅行	18
北西	18
国際協力	18
べる。	18
乾燥	18
オリックス	18
涙	18
消防車	18
守口市	18
バラバ	18
鉢	18
たらい	18
機関委任事務	18
こなし	18
国防	18
競争力	18
レンタカー	18
名人	18
保育	18
おしゃれ	18
井上	18
応用	18
ひび	18
沼	18
さわやか	18
六分	18
墨	18
青春	18
菜	18
ブル	18
プリン	18
代表質問	18
却下	18
賛否	18
経済制裁	18
マー	18
淀川区	18
八幡	18
凶	18
打球	18
長打	18
併殺	18
抑止	18
ソーラ	18
新生	17
拒否権	17
入植	17
寛容	17
学問	17
アウト	17
ボーナス	17
公共事業	17
コラム	17
スコア	17
許さない	17
同胞	17
クローズアップ	17
会費	17
男らしさ	17
肯定	17
北陽	17
プレス	17
新田	17
岡山	1

肥大	7
忠誠	7
あぶく	7
伝承	7
いたる	7
きぬ	7
ルーキー	7
平凡	7
セットプレー	7
ゴールキック	7
瑞穂	7
ケーブルテレビ	7
全国紙	7
携帯	7
川底	7
諏訪	7
日通	7
ガンバ	7
五台	7
陳列	7
作家	7
ナビ	7
パレード	7
退屈	7
デリ	7
ケート	7
国民感情	7
政党交付金	7
政府代表	7
武力衝突	7
口コミ	7
ミネラル	7
正体	7
降水量	7
ネオン	7
シャープ	7
セメント	7
国中	7
洗練	7
アトリエ	7
手帳	7
おむつ	7
レベルアップ	7
キックオフ	7
三洋電機	7
フィル	7
アレンジ	7
エンディング	7
ポリ	7
潔癖	7
港湾局	7
厄年	7
池袋	7
引き分け	7
うず	7
官報	7
取り直し	7
フロア	7
飯塚	7
佐渡	7
サント	7
新星	7
ゆい	7
レッスン	7
裏切り	7
床面積	7
民主政	7
侵略戦争	7
有価証券	7
符	7
学習塾	7
オーク	7
西村	7
歳入	7
トランペット	7
二人組	7
大統領	7
業務用	7
升	7
義理	7
返信	7
タイムリー	7
張り紙	7
小中学校	7
松戸市	7
測量	7
地盤沈下	7
陰謀	7
正論	7
配合	7
発散	7
十八番	7
委譲	7
規範	7
試食	7
ボーカル	7
屋外	7
麻酔	7
受託	7
ベルギー	7
職務質問	7
協和	7
出	7
大同	7
非	7
地方都市	7
過失	7
かんな	7
はぜ	7
氷河	7
義和	7
マス	7
充電	7
会いたい	7
大目	7
輸入車	7
身代	7
人事異動	7
ニコニコ	7
大黒	7
意表	7
ジャパンエナジー	7
慶	7
公共料金	7
宝物	7
陣地	7
藤沢市	7
閲覧	7
峠	7
石油化学	7
本家	7
いいじゃない	7
あなたに	7
辞典	7
十二時	7
死亡率	7
電算	7
パイ	7
腐食	7
迎合	7
遺伝	7
人民	7
特別仕様車	7
文房具	7
へのこ	7
燃焼	7
ラップ	7
石橋	7
川口	7
心中	7
花道	7
数字選択式全国自治宝くじ	7
酵素	7
プレジャーボート	7
血圧	7
オーナー	7
緊急避難	7
試験車	7
法定	7
テニス	7
文化センター	7
駒沢	7
ハート	7
在来線	7
巡礼	7
ショート	7
フェア	7
インテリア

ホク	4
洋酒	4
学校行事	4
ドンド	4
プレイ	4
スパイ	4
同人	4
町田市	4
カフェ	4
セキ	4
集中豪雨	4
実数	4
パチ	4
ゴーストタウン	4
防潮堤	4
王将	4
六湛寺町	4
債権回収	4
空転	4
四島	4
省令	4
スピードアップ	4
大動脈	4
コンビナート	4
天然ガス	4
一極集中	4
動議	4
艦	4
大規模開発	4
確信犯	4
量刑	4
修羅場	4
エラー	4
ストライク	4
マイペース	4
ステッカー	4
関根	4
李	4
めぐみ	4
美学	4
米倉	4
ボレー	4
アタック	4
フジタ	4
草加	4
ドド	4
フライパン	4
ターミナルケア	4
の宗教	4
生息地	4
新居	4
拒絶反応	4
本村	4
圧縮	4
紅梅	4
礫	4
オカ	4
真夜中	4
ボン	4
山菜	4
幕府	4
農作物	4
土地区画整理事業	4
ルック	4
西行	4
姫島	4
湾岸線	4
ボーイスカウト	4
福井県	4
みやこ	4
福山市	4
吉田町	4
岡町	4
運動場	4
松前町	4
広田	4
富岡	4
サイレン	4
大蔵省	4
繰り上げ当選	4
尾崎	4
教諭	4
型式	4
初等教育	4
歌姫	4
大沼	4
胃	4
大量破壊兵器	4
凶作	4
ベア	4
寿町	4
桜町	4
曽根	4
宮町	4
原字	4
森町	4
刈谷市	4
垂水	4
富田	4
千代田町	4
中京区	4
寺町	4
鳥羽	4
小字	4
寺田	4
朝日町	4
村井	4
大町	4
土浦市	4
千束	4
並木	4
麻生	4
浅間	4
プラザ	4
戸田市	4
西原	4
磯子区	4
洋光台	4
代田	4
高松町	4
西山町	4
四番町	4
新在家	4
加茂	4
栗原	4
古城	4
手話	4
ジン	4
年上の女	4
問題意識	4
スラッガー	4
マスク	4
虎ノ門	4
の地下鉄	4
超法規的措置	4
貨物列車	4
睡眠	4
地理	4
ボランティア元年	4
男役	4
珠	4
マリ	4
キリスト教	4
買い占め	4
一平	4
閉会	4
スーパーマーケット	4
アルファベット	4
ジア	4
マンチェスター	4
クリスタル	4
ドリー	4
八日目	4
カタ	4
からくり	4
ビオ	4
立山	4
判決理由	4
五十戸	4
超音波	4
共同購入	4
帝国	4
近代詩	4
篆刻	4
判事	4
小久保	4
総務

スワップ	2
貝取	2
実況中継	2
びわ湖毎日マラソン	2
性癖	2
情報操作	2
百選	2
行水	2
シュン	2
自活	2
マネーゲーム	2
内部情報	2
リクルート	2
念仏	2
林一	2
ぐうたらママ	2
休会	2
非常事態宣言	2
社会政策	2
運転者	2
金融商品	2
労働協約	2
レイ	2
ジョブ	2
格付け機関	2
インナ	2
高校三年生	2
固定観念	2
婚姻関係	2
内部崩壊	2
軍団	2
レフ	2
王様	2
きらい	2
キャスティ	2
応援歌	2
イメージソング	2
西脇唯	2
高速バス	2
告訴状	2
郵便受け	2
完成予想図	2
土蔵	2
三浦郡	2
長柄	2
南平台	2
中目黒	2
細田	2
成功報酬	2
パーク	2
折り紙	2
絵馬	2
和菓子	2
舞姫	2
下河原	2
住吉山手	2
御影町	2
大蔵	2
衣川	2
公布	2
なごや	2
予定価格	2
寝台車	2
一時停止	2
庇	2
今を生きる	2
高学年	2
索引	2
民話	2
チャペル	2
ブライダ	2
弦楽四重奏	2
放流	2
稚魚	2
開帳	2
ワイングラス	2
タイトル・ロール	2
ホロ	2
アリーナ	2
フィナーレ	2
ミディアム	2
バンバン	2
小学一年生	2
ポカ	2
平年値	2
柏崎市	2
青山町	2
茶道	2
国公立大学	2
インドネシア	2
精進	2
役者絵	2
版元	2
日本浮世絵協会	2
王立	2
ブン	2
ケット	2
公用語	2
神殿	2
鼎	2
パラレル	2
地方六団体	2
リベラル	2
経済企画庁	2
心を開いて	2
おねえさん	2
非人道的	2
性差別	2
通貨同盟	2
おだいじに	2
魔法	2
加藤光	2
おでん	2
原典	2
大吉	2
憲法改正	2
自由港	2
旅客	2
縁組	2
ボブ	2
ジョン	2
ブライ	2
プラス思考	2
火の粉	2
金融センター	2
過積載	2
つけば	2
岩城	2
ダークホース	2
バロメーター	2
怪力	2
歩数計	2
チームワーク	2
風の影	2
ゼク	2
難波英樹	2
山口大輔	2
雪男	2
クロスカントリースキー	2
鉱業	2
ラフト	2
別所哲也	2
プライムタイム	2
磁器	2
陶磁器	2
鵠沼	2
三町	2
御殿山	2
門川町	2
かわいがり	2
発言権	2
嵐	2
混獲	2
マイアミ	2
脱獄	2
ホー

ダイレクトメール	2
複写機	2
画像診断	2
海藻	2
シャボン玉	2
団体戦	2
アサ	2
青井	2
知念	2
上四方固	2
今川	2
李小双	2
田中光	2
西川大輔	2
バーガー	2
パワーアップ	2
画像処理	2
動く歩道	2
里山	2
迎賓館	2
因子	2
は特融	2
三金	2
キャピタル	2
西天満	2
アポトーシス	2
長居	2
鹿島市	2
損害保険	2
お色気番組	2
電話網	2
注意書き	2
ケント	2
醸造酒	2
環境音楽	2
詩篇	2
光と影	2
クイーン	2
薬害	2
裁断	2
傾斜計	2
火山ガス	2
浮力	2
大量虐殺	2
戸井田三郎	2
専門部	2
国際犯罪	2
薬価	2
電磁波	2
レント	2
わが町	2
嫌み	2
烏龍	2
茶々	2
調剤	2
四輪車	2
レンタル	2
ミューチュアル・ファンド	2
おしぼり	2
ボイル	2
クワ	2
唐津	2
土下座	2
ミロ	2
段違い平行棒	2
平均台	2
菅原リサ	2
橋口美穂	2
大川真澄	2
ジュビロ磐田	2
鈴木秀	2
相馬	2
ドゥンガ	2
大嶽	2
ジェニ	2
ジーニョ	2
柏レイソル	2
土肥	2
加藤望	2
ロタ	2
ジェフ	2
リックス	2
サントス	2
ペレイラ	2
井原	2
丹波哲郎	2
ニ長調	2
弦楽四重奏曲	2
罷免	2
ガマ	2
蛍光灯	2
美濃	2
掛川	2
小松島	2
野高	2
及川	2
宮下	2
キューバ	2
二郎	2
北海道拓殖銀行	2
嵯峨	2
御坊	2
大玉	2
箕島	2
興国	2
芝高	2
志度	2
尾道	2
筑紫	2
いつものように	2
指名停止	2
電話帳	2
上質紙	2
国際規格	2
漫画雑誌	2
取次	2
無菌	2
かすり	2
勇み足	2
輪廻	2
りんね	2
コーリング	2
多面体	2
サブタイトル	2
大学祭	2
少女漫画	2
国家試験	2
ブラックホール	2
星雲	2
公告	2
コミットメント	2
アナ	2
使用者	2
不夜城	2
頭痛	2
海底油田	2
ピックアップトラック	2
司法取引	2
はしご酒	2
か焼	2
環境ビジネス	2
勅使川原郁恵	2
ジャン	2
丸山茂樹	2
関西大学	2
石塚	2
アシスト	2
東伏見アイスアリーナ	2
オッズ	2
運動公園	2
大徳	2
ドレ	2
サズ	2
イブキ	2
スクーター	2
工具	2
島

鑿	1
常民	1
道路標識	1
彷徨	1
大名行列	1
駕籠	1
土用	1
寒熱	1
幼名	1
同格	1
領地	1
ムシ	1
とり天	1
嫡出	1
回り道	1
サリー	1
ゲテモノ	1
苦味	1
辛味味	1
すみや	1
厚化粧	1
タラーリ	1
淡水魚	1
ョ	1
粘液	1
大発見	1
生活史	1
自動券売機	1
山陽新幹線	1
新大阪	1
東海道線	1
阪急伊丹線	1
阪急甲陽線	1
阪神本線	1
鈴蘭台	1
有馬温泉	1
姫路バイパス	1
防犯灯	1
神楽町	1
クラック	1
普通列車	1
消火栓	1
火事場泥棒	1
プリペイドカード	1
一枚の切符	1
兵庫中学校	1
宗茂	1
中村茂雄	1
呉竹	1
広陵町	1
引田町	1
那智	1
東畑	1
堀宏	1
小織	1
お世話になりました	1
南消防署	1
おはなはん	1
東小学校	1
セサル	1
奈良病院	1
安食	1
帝塚山	1
天命	1
野上町	1
やすらぎ	1
親和会	1
御浜町	1
橋本昌	1
青山浩志	1
斉藤満	1
林啓子	1
遠藤仁	1
東予市	1
岡田栄	1
神戸大空襲	1
上田誠一	1
更生保護	1
松丸	1
山田秀	1
御坊市	1
津江	1
香奈	1
上牧町	1
富原	1
前田喜史	1
花井	1
石田博三	1
太田茂	1
三浦貴	1
ピーズ	1
井上智恵	1
中宗	1
勝子	1
平島	1
中島正	1
アキノ	1
高山正之	1
保存会	1
高橋光	1
文聖	1
高橋滋	1
高橋正	1
小林正人	1
加藤精三	1
矩	1
井上清一	1
江東	1
唯	1
鯨岡	1
国分寺町	1
ユベール	1
喜多村	1
清水晶	1
藤原啓	1
日本民族	1
土門	1
秀和	1
岡幸恵	1
中村充孝	1
森弘	1
杉山洋子	1
太田裕美	1
中島清	1
桐畑	1
菊地慶	1
中川理恵	1
中野裕子	1
第四中学校	1
鈴木八郎	1
吉村紀子	1
ファミリースキー	1
津留	1
大橋仁	1
大阪府立寝屋川高等学校	1
バザール	1
大阪市立豊崎小学校	1
風邪	1
まゆみ	1
ネオ	1
中川淳	1
クエ	1
池田町	1
中村正	1
東浦	1
オペレーション	1
岡村茂	1
麻衣	1
大仏	1
谷口拓也	1
膳所	1
山脇	1
山東町	1
恭平	1
倉吉市	1
豊中市立蛍池小学校	1
山崎裕	1
土岐	1
斉藤明	1
可

有森浩三	1
井上慶太	1
長沢千和子	1
鹿野圭生	1
加藤一二三	1
児玉孝一	1
中田功	1
酒井順吉	1
林葉直子	1
多目的ホール	1
佐藤康光	1
長谷部久雄	1
中田宏樹	1
真田圭一	1
三浦弘行	1
深浦康市	1
植村真理	1
くすのき台	1
宮田利男	1
小林宏	1
谷川治恵	1
名人戦	1
テレコム	1
ニフティサーブ	1
脇謙二	1
今泉健司	1
玉名市	1
蓮華院	1
福崎文吾	1
増田裕司	1
赤坂プリンスホテル	1
中村修	1
西浦温泉	1
安用寺孝功	1
関根茂	1
青野照市	1
菰野町	1
湯の山温泉	1
向陽	1
山本真也	1
膿	1
院内感染	1
映画評論	1
千羽鶴	1
たかみ	1
ノンプライム	1
汽車	1
夜逃げ	1
帝国軍	1
フグ	1
風が吹いている	1
スポーツ新聞	1
清め	1
富士見書房	1
交響組曲	1
電極	1
電光掲示板	1
片町	1
柴崎町	1
上連雀	1
深大寺	1
人にやさしく	1
強制執行	1
玉緒	1
第二小学校	1
中田茂	1
松本博	1
江口	1
トライアスロン	1
法華宗真門流	1
正広	1
教育研究	1
リアン	1
焼肉	1
滑稽	1
福泉	1
観音寺市	1
観音寺	1
青木薫	1
洋画家	1
水車	1
大旗	1
一本打ち	1
教授会	1
不当解雇	1
摩天楼	1
古代都市	1
軽減税率	1
社団	1
手広	1
東風	1
集団主義	1
ディエゴ・マラドーナ	1
カパン	1
カノ	1
血圧計	1
マラドーナ	1
酔っ払い	1
扇子	1
性愛	1
公開対局	1
全中	1
オムライス	1
シュシュ	1
ウーロン	1
つばめグリル	1
ドトールコーヒー	1
フォーエバー	1
ロフト	1
京葉線	1
舞浜	1
東京ディズニーランド	1
ポテトチップス	1
片倉工業	1
時がたてば	1
社史	1
アクバル	1
挙国一致内閣	1
南勢	1
北勢	1
再統一	1
ワードプロセッサ	1
元気ですか	1
昭和維新	1
戒厳	1
随伴	1
カラカラ	1
壮年	1
無条件降伏	1
国史	1
ファースト	1
典礼	1
聖歌	1
悲しみの聖母	1
直訳	1
コンパス	1
ランドマーク	1
ニール	1
ソンズ	1
夏の夜の夢	1
鬼瓦	1
道成寺	1
アンフォゲッタブル	1
ナット・キング・コール	1
クラーク	1
知

マルタ	1
侵華日軍第七三一部隊罪証陳列館	1
実験動物	1
ボイラー室	1
給湯設備	1
絶版	1
手形貸付	1
兄と妹	1
校歌	1
府道	1
鉄パイプ	1
地元新聞	1
校正	1
精霊	1
人海戦術	1
世界文化	1
新高	1
、お前もか	1
城米	1
中南海	1
敵前逃亡	1
反逆児	1
伊藤一	1
村井温	1
伊藤哲朗	1
正和	1
万寿	1
柳沢	1
きみ	1
大臣	1
秘書官	1
岐阜県	1
ジブチ	1
福田博	1
中島敏次郎	1
藤森昭一	1
宮内庁	1
長尾立子	1
社会福祉協議会	1
の声	1
駐	1
禁忌	1
捨て子	1
もっと…	1
ワークシェアリング	1
機動捜査隊	1
六合	1
反核団体	1
第二地方銀行	1
証文	1
ウトウ	1
ペンチ	1
はんだ	1
万力	1
調味料	1
判例集	1
成人向け	1
列強	1
学習者	1
労働力調査	1
絶対服従	1
修身	1
シュウ	1
アレコ	1
個人攻撃	1
間接選挙	1
大首長	1
ラーゲリ	1
第三の道	1
粛清	1
リミット	1
ての経済	1
二つの	1
ボスニア	1
ミニディスク	1
粒子	1
マイレージサービス	1
無段変速機	1
電化	1
周辺機器	1
瓶詰	1
経営哲学	1
品質管理	1
兵庫銀行	1
信用取引	1
マイクロフィルム	1
数値制御	1
制御装置	1
借款	1
農業技術	1
帯封	1
箪笥町	1
神楽河岸	1
個別指導	1
点字図書	1
柳町	1
感覚器	1
いっぱい	1
ごめんね	1
吉田豊	1
小宮山	1
相乗効果	1
代走	1
日本新薬	1
小西酒造	1
平野雅人	1
田代	1
井本直歩子	1
粕谷	1
センターコート	1
相撲教習所	1
ジミー	1
コルデロ	1
七色	1
愚行	1
保護地域	1
鳥獣保護区	1
自然保護団体	1
逮	1
西浅草	1
辻堂東海岸	1
学童保育	1
南大泉	1
風景写真	1
風になって	1
上場廃止	1
ファクター	1
高分子	1
ポリエステル	1
有事	1
ウォーク	1
ハイウェイ	1
さつま	1
将棋まつり	1
日本棋院	1
夜市	1
カブトムシ	1
桜小学校	1
アンテナショップ	1
誠心誠意	1
千原町	1
割れた窓	1
シャーン	1
南の海	1
中野駅	1
広葉樹	1
捕食	1
乱婚	1
茶飯	1
重婚	1
首切り	1
タ

荒井良	1
吉野光	1
清和	1
高村	1
宗論	1
国栖	1
日比谷公会堂	1
歌丸	1
立川流	1
立川志らく	1
真打	1
ルイス	1
ライオンと魔女	1
マルセ太郎	1
黄昏	1
結婚物語	1
春風ひとみ	1
夏夕介	1
自由人	1
栗山民也	1
青年団	1
弘前劇場	1
平田オリザ	1
ポップ・シー	1
グラマラス	1
オリジナル・サウンドトラック	1
ガール・グループ	1
シンクロ	1
文通	1
からっと	1
ルイ	1
パステルナーク	1
マイケル・ナイマン	1
ナイマン	1
英国式庭園殺人事件	1
サントリーホール	1
アシュ	1
バーバー	1
きよしこの夜	1
コラール前奏曲	1
コレル	1
パルティータ	1
フルート協奏曲	1
コブシ	1
韻	1
男の子と女の子	1
蘇生	1
噴水	1
フラー	1
約束手形	1
継電器	1
プログラミング言語	1
政府の失敗	1
鑑定人	1
ら間	1
た特融	1
お蔵入り	1
メロドラマ	1
在・	1
社会的地位	1
皇国	1
マジョリティ	1
民族学	1
虜	1
マイナー	1
セレ	1
ベス	1
若き	1
挽歌	1
歓送迎会	1
秘話	1
アイマックス	1
暗闘	1
ポーカー	1
大当たり	1
三教	1
石油ストーブ	1
敬老優待乗車証	1
家庭裁判所	1
在留期間	1
タクシ	1
空車	1
破産債権	1
生きとし生けるもの	1


全体的な感想:

* 曖昧さ回避ページが多い
* 固有名詞も結構多いように感じる

固有名詞が多いのはどういうことなんだろう？

* 正解データの質が悪い?
  * 辞書ベースではついたのに正解にはついていない
* 正解データの読み込み方にバグがある?
* `to_iob2`関数でignoredを弾いたのに問題がある?

おそらく、`to_iob2`関数に原因がある。というわけで修正して再びやってみる。

In [28]:
X_true, y_true = to_iob2(mainichi_dir, {})
y_pred = [recognizer.iob2(text) for text in X_true]

chars = []
tags = []
for pred, true, doc in zip(y_pred, y_true, X_true):
    for p, t, char in zip(pred, true, doc):
        if t == 'O' and p != 'O':
            chars.append(char)
            tags.append(p)
            
entities = get_entities(tags)

words = []
for _, begin_offset, end_offset in entities:
    end_offset += 1
    word = ''.join(chars[begin_offset: end_offset])
    words.append(word)
    
freq = Counter(words)
freq100 = sum(v for _, v in freq.most_common(n=100))

print('Num of tokens: {}'.format(len(words)))
print('Num of types  : {}'.format(len(set(words))))
print('{} / {}'.format(freq100, len(words)))

for word, fr in freq.most_common():
    print('{}\t{}'.format(word, fr))

Num of tokens: 1543995
Num of types  : 14200
1156096 / 1543995
の	108630
に	65662
を	56701
は	52035
い	51308
が	50957
る	48821
と	48303
で	46448
た	43462
て	42759
な	34610
し	27673
っ	26002
れ	24157
か	23814
ら	21535
も	20748
す	18923
だ	15706
した	15399
こ	14967
う	14882
り	14878
ま	12186
く	11548
め	10967
き	10834
け	10432
ど	10402
あ	10011
さ	9897
え	8221
そ	7668
や	6961
ん	6117
つ	5948
わ	5486
ち	4148
ろ	4033
ば	3982
的	3909
よう	3692
み	3660
よ	3605
大	3526
せ	3485
行	3330
年	3144
中	3091
ず	2973
お	2921
じ	2914
べ	2880
一	2790
へ	2777
人	2690
時	2496
げ	2333
方	2320
要	2296
しい	2295
はな	2288
対	2210
内	2077
分	2038
取	2033
ほ	2026
力	1982
受	1979
強	1957
場	1951
高	1947
目	1827
とも	1809
初	1804
実	1787
地	1700
間	1692
下	1681
通	1674
む	1669
合	1669
かし	1651
手	1618
び	1610
生	1608
新	1599
調	1583
国	1582
話	1580
日	1566
自	1509
元	1493
金	1492
全	1484
問題	1424
明	1420
本	1394
面	1335
体	1322
また	1321
政府	1282
ご	1275
向	1260
なかった	1252
かな	1215
私	1212
書	1200
ぶ	1183
化	1174
支	1165
より	1124
ね	1116
主	1106
定	1101
関係	1100
直	1093
気	1087
連	1081
反	1062
変	1047
いの	1038
今年	1034
何	1024
まれ	1012
数	100

動機	49
体力	49
現役	49
再生	49
会話	49
同期	49
上下	49
障害	49
カギ	49
トイ	49
草	48
強力	48
当事者	48
秘	48
諮問	48
殺し	48
裁判所	48
糸	48
たたき	48
プロジェクト	48
農家	48
ファッション	48
叫	48
善	48
ストレス	48
シェア	48
セット	48
恐怖	48
さわ	47
崩し	47
分裂	47
市町村	47
ゅ	47
武	47
民営化	47
ルール	47
敬	47
つながり	47
赤	47
中盤	47
コンクリート	47
停戦	47
スタイル	47
ペー	47
家屋	47
上場	47
同意	47
車両	47
テープ	47
遺憾	47
製作	47
あき	46
もん	46
ピーク	46
移民	46
怒	46
競技	46
拍手	46
ゲリラ	46
掃除	46
主人公	46
ミット	46
携帯電話	46
撮	46
郵便振替	46
安打	46
排除	45
下り	45
南北	45
西側	45
予約	45
動員	45
大丈	45
資源	45
通報	45
皮	45
看板	45
毒	45
解体	45
軟禁	45
貧困	44
後退	44
補償	44
未来	44
青年	44
徐	44
設備投資	44
リズム	44
名誉	44
ティー	44
くわ	44
レストラン	44
脱出	44
給与	44
解雇	44
休日	44
劇場	44
植物	44
輪	44
宗教団体	44
巡回	44
運賃	44
古代	44
連勝	44
虚	44
薬品	44
わたり	44
勇気	44
露	44
治安	44
プラント	44
回避	44
本店	44
拘束	44
大震災	44
仮設	44
連立政権	43
電子	43
モノ	43
指標	43
素材	43
文学	43
統計	43
演技	43
刑事	43
重さ	43
キャンプ	43
コン	43
倍	43
はね	43
留学	43
軌道	43
軒	43
マルチメディア	43
告示	43
面的	43
協賛	43
園	43
補正予算	43
体質	42
妥協	42
未定	42
聞こえ	42
咲	42
家事	42
理想	42
故障	42
執筆	42
登山	42
はるか	42
尋	42
地上	42
衝撃	42
定員	42
電力	42
観客	42
プレゼント	42
防御	42
ダメ	42
サイン	

打ち切り	14
愛称	14
休止	14
品質	14
シア	14
チブ	14
性質	14
損傷	14
保育所	14
ハブ	14
背番号	14
喫茶店	14
かしこ	14
備蓄	14
あの日	14
観光地	14
眼鏡	14
歌詞	14
運搬	14
一色	14
断言	14
陥落	14
パネルディスカッション	14
裁決	14
振動	14
注射	14
衝	14
相乗り	14
曇り	14
覚書	14
遷都	14
カビ	14
拡散	14
上映会	14
バッテリー	14
公益法人	14
養父	14
遺構	14
モーター	14
石室	14
朝刊	13
の政治	13
次元	13
毛	13
正義	13
邪魔	13
海軍	13
長期金利	13
中央銀行	13
金融市場	13
呼吸	13
装飾	13
チャリティー	13
パスポート	13
身分	13
下町	13
無意識	13
ニック	13
からし	13
顧	13
メダル	13
ネック	13
氷	13
さび	13
ティン	13
乾	13
供養	13
サラ	13
警報	13
助言	13
緑	13
レジャー	13
スポット	13
操縦	13
舞踊	13
食卓	13
包囲	13
後悔	13
平面	13
王国	13
オーバー	13
初日	13
活字	13
化合物	13
停電	13
議長	13
学部	13
研究室	13
宮	13
定番	13
ルー	13
適当	13
配管	13
恋愛	13
渦	13
国立	13
最強	13
起点	13
足場	13
監禁	13
攻略	13
北西	13
ソプラノ	13
名物	13
クリーン	13
マスメディア	13
ペン	13
全会一致	13
大口	13
休業	13
原油	13
市場開放	13
申込	13
メジャー	13
電話相談	13
景観	13
波紋	13
憶測	13
暖房	13
硬派	13
ノリ	13
サオ	13
おろし	13
メロディ	13
焼死	13
分散	13
削除	13
おとな	13
エリート	13
自然環境	13
物流	13
電報	13
洗剤	13
ユーモア	13
生態系	13
三味線	13
診療	13
畳	13
合理	13
やらせ	13
赤字国債	13
渦巻	13
主軸	13
書店	13
バッタ	13
分子	13
重火器	13
ヒアリング	13
ブル	13
握手	13
わに	13
角度	13
日開	13
博

結晶	5
フィーバー	5
封じ手	5
バチ	5
重宝	5
トリオ	5
嘱託	5
皇族	5
審査会	5
司法解剖	5
バケツ	5
広告代理店	5
ツル	5
聖火	5
降雪	5
無口	5
日食	5
征服	5
熱帯雨林	5
遠足	5
鉱山	5
在野	5
代名詞	5
自我	5
後	5
足掛け	5
ライセンス	5
舵	5
海岸線	5
銅	5
媒介	5
亜熱帯	5
渓流	5
日用品	5
織物	5
稲作	5
自己主張	5
北へ	5
川岸	5
交通規制	5
啓	5
定款	5
運河	5
水草	5
帰りたい	5
売春	5
検問	5
初日の出	5
ハッピー	5
連載小説	5
電気通信事業者	5
分業	5
改編	5
戦争責任	5
ラッシュ時	5
高度経済成長	5
暗号	5
劣等感	5
四角	5
配列	5
図形	5
連作	5
オロ	5
きぬ	5
効用	5
さらし	5
ホープ	5
ハンドリング	5
セーブ	5
昔話	5
フロッピー	5
の女	5
試薬	5
山小屋	5
国宝	5
重要文化財	5
レトロ	5
パレード	5
強制収容所	5
自首	5
密室	5
短命	5
退屈	5
親切	5
仲裁	5
渇水	5
水槽	5
アルカリ	5
リバウンド	5
電源	5
煮物	5
核家族	5
平均寿命	5
破局	5
内観	5
競技場	5
威信	5
休場	5
竜巻	5
ノーサイド	5
原文	5
引き分け	5
離陸	5
オヤ	5
歌謡曲	5
のぶ	5
謡	5
交通費	5
脅迫状	5
最低賃金	5
拷問	5
針金	5
消防団	5
オフィス街	5
名札	5
居場所	5
熱水	5
骨抜き	5
政治システム	5
国民投票	5
マイホーム	5
あっと	5
潤	5
政官	5
黄信	5
軍事費	5
南国	5
プライムレート	5
内在	5
地方財政	5
いのこ	5
転々	5
古本	5
ルポ	5
新星	5
リリーフ	5
財政再建	5
運動量	5
命日	5
カール	5
体外受精	5
カモ	5
先祖	5
ブルドーザー	5
独学	5
鎮魂	5
の人	5
貴族	5
体温	5
エディ	5
売店	5
花瓶	5
工房	5
リーズ	5
スタジアム	5
ワールド	5
サント	5
ビート	5
知能	5
リベート	5
アブ	5
同時代	5
テキスト	5
国語	5
語尾	5
舌鼓	5
下降	5
価格競争	5
大同	5
全滅	5
キャス	5
暴言	5
株式会社

手綱	2
計数	2
晩婚化	2
予防外交	2
ラリス	2
ボーテ	2
大きな政府	2
下野	2
真空地帯	2
スパ	2
みなみ	2
爆弾	2
傍受	2
海軍基地	2
新たな時代	2
社債	2
教育制度	2
凪	2
衆参同日選挙	2
針路	2
シラ	2
擬態	2
溢れる	2
モラ	2
頭脳流出	2
拳法	2
新雪	2
性的虐待	2
新しい波	2
太陽	2
勝負師	2
ピックアップ	2
おはよう	2
オク	2
繊	2
両輪	2
お七夜	2
お印	2
冬型	2
断筆	2
空襲警報	2
蔵元	2
年賀	2
リセ	2
戦中	2
防寒着	2
ララ	2
氷床	2
ボラ	2
砂塵	2
さじ	2
もっと強く	2
豪農	2
病院船	2
水平線	2
刺激臭	2
ハプニング	2
思春期	2
民芸品	2
カーン	2
レトルト	2
固有種	2
寄宿舎	2
島国	2
馬賊	2
名士	2
偉人	2
そうかな	2
エキゾチシズム	2
エス	2
ピオ	2
熱気球	2
魅せられて	2
フィットネス	2
マスト	2
ナビゲーター	2
スターボー	2
ジャイブ	2
マニューバ	2
クローズ	2
水差し	2
三拍子	2
微風	2
潮干狩り	2
世界一周	2
植生	2
マリン	2
船尾	2
水上	2
哀愁	2
化学繊維	2
大洪水	2
めっき	2
水力発電	2
実質所得	2
有機農業	2
熱風	2
野良	2
異文化	2
陳修	2
原野	2
油圧	2
コンコース	2
雨戸	2
過密	2
絵	2
素描	2
通信販売	2
高齢者福祉	2
位相	2
鉄のトライアングル	2
リーディング	2
玉虫色	2
新エネルギー	2
反車	2
蛇口	2
男湯	2
行政指導	2
金型	2
ザ・ベスト	2
天と地	2
姪	2
腕時計	2
児童画	2
ブロン	2
どうしてだろう	2
ジョイ	2
節税	2
余暇	2
出前	2
万年	2
カルチャーセンター	2
なば	2
ミドルシュート	2
ガンガン	2
ハイボール	2
切り込み	2
近未来	2
静止軌道	2
時事問題	2
鉄塔	2
お任せ	2
ダンスホール	2
密葬	2
ラジオドラマ	2
リズ	2
ミカル	2
金粉	2
アウトサイダー	2
西洋	2
美術教育	2
個人タクシー	2
生意気	2
色素	2
爆竹	2
初売り	2
離着陸	2
大観	2
縁日	2
シック	2
大義	2
秘文	2
自己

一文字	1
ホームタウン	1
ゲームメーカー	1
奇襲	1
肉弾	1
聴取率	1
条	1
コーポレーション	1
有刺鉄線	1
定公	1
北尾	1
ボーダー	1
アカデミック	1
や会	1
冷害	1
旅客ターミナル	1
日満	1
アンチ	1
政治資金パーティー	1
属性	1
有為	1
地方名	1
水入り	1
湯冷まし	1
ろ過	1
カルキ	1
沸点	1
回診	1
理工学	1
脱水	1
無尽蔵	1
プライベートブランド	1
ライフサイクル	1
上地	1
論証	1
女性自身	1
ミスコ	1
フェミニン	1
ナジ	1
エリー	1
上書	1
もってのほか	1
ファイン	1
ロース	1
だま	1
ゼッケン	1
夢のひとつ	1
ぼくら	1
ホイッスル	1
ノット	1
リリース	1
ロックス	1
人材開発	1
制作プロダクション	1
で放	1
反応機構	1
水溶液	1
中川和雄	1
総理	1
プロパー	1
中央線	1
ラデ	1
管制塔	1
コロコロ	1
逆噴射	1
みの駅	1
連絡網	1
花札	1
忘却	1
シマ	1
がぶり	1
新年会	1
フォークソング	1
トーク番組	1
送信機	1
トランス	1
先読み	1
病み上がり	1
企業戦士	1
チー	1
セックスレス	1
シャリ	1
国外退去	1
犯行声明	1
目には目を	1
無価値	1
鉄鉱石	1
文明の衝突	1
アナーキー	1
好色	1
改姓	1
シュラ	1
正統派	1
読み切り	1
結婚願望	1
中央分離帯	1
ガードレール	1
大使	1
子どもの権利	1
思想・良心の自由	1
万歳	1
スペア	1
バリン	1
自然科学	1
管区気象台	1
エクイティ	1
コモ	1
ディティ	1
スワップ	1
シリアス	1
出欠	1
る会見	1
ねじれ現象	1
永年	1
グランドデザイン	1
烈火	1
三分	1
農地転用	1
他人の家	1
テレビ中毒	1
何度でも	1
空売り	1
インサイダー	1
自業自得	1
宗教問題	1
経営企画	1
基本粒子	1
ヒッグス粒子	1
巨大科学	1
基礎研究	1
基礎科学	1
ただ乗り	1
反感情	1
いつのまに	1
互換機	1
ワークステーション	1
海運	1
定先	1
混沌	1
スローダウン	1
オーバーキル	1
ダック	1
レディース	1
おはじき	1
お手玉	1
すごろく	1
福笑い	1
戦災孤児	

人間魚雷	1
タグボート	1
災害食	1
別所	1
凡人	1
寝酒	1
仮名遣い	1
持久戦	1
鉱業	1
貴族院	1
親派	1
行政処分	1
外務省	1
スンナ	1
ナンパ	1
地租	1
てんま	1
公正取引	1
基地局	1
交換機	1
販売時点情報管理	1
議員連盟	1
狂想曲	1
た市	1
巡察	1
町人	1
独裁者	1
暴力装置	1
リス	1
ダンスパーティー	1
産み分け	1
魚道	1
統治機構	1
足尾銅山	1
鉱毒	1
再配分	1
型システム	1
原子力船	1
単発	1
古書店	1
書写	1
名画座	1
夜行列車	1
幻惑	1
運命のい糸	1
地域研究	1
情報革命	1
精神史	1
宗教改革	1
原典	1
はじまり	1
竹細工	1
水引	1
循環系	1
奇人	1
太郎	1
早替わり	1
湖沼	1
生物化学的要求量	1
生活排水	1
海洋汚染	1
水質汚濁	1
火力発電所	1
環境計画	1
ナンバープレート	1
環境技術	1
健康障害	1
トラフ	1
クトロ	1
メーター	1
スペクトル	1
気候変動	1
結合エネルギー	1
元素合成	1
軟派	1
この街で	1
ホールインワン	1
長崎	1
ポールポジション	1
リカバリー	1
硫黄	1
相撲甚句	1
えび	1
自己分析	1
台記	1
代行	1
フラストレーション	1
落馬	1
重賞	1
馬主	1
着差	1
競走中止	1
花いっぱい運動	1
街路	1
アラカルト	1
グラマー	1
西高東低	1
反比例	1
田単	1
作人	1
豪雪地帯	1
臨床実習	1
本当の恋	1
玉の輿	1
嫌な女	1
マニ	1
銀河	1
ピオネール	1
高等裁判所	1
欠席裁判	1
広島	1
歳費	1
省庁制	1
双眼鏡	1
息子たち	1
蛍火	1
テレビアニメ	1
和尚	1
トイレタリー	1
営業職	1
ブルーカラー	1
企業研究	1
無国籍	1
セクター	1
召集令状	1
ストーン	1
製版	1
一夜城	1
大きな水たまり	1
使用貸借	1
小波	1
林野	1
間接証拠	1
マスゲーム	1
ニアピン	1
メッカ	1
オーバーホール	1
雨上がり	1
台湾料理	1
でんでん	1
指人形	1
ニコ	1
いて座	1
マヌ	1
鳥葬	1
紀元前	1
れんが	1
段丘	1
の空襲	1
山門	1
台座	1
近代文学	1
体育大学	1
管理

B-ignoredを考慮したことで、固有名詞を結構取り除けた。

さらに精錬する方法は？

* 形態素解析で分かち書きした境界に一致するか判定